In [1]:
import spiketoolkit as st
import spikesorters as ss
import spikeextractors as se
import numpy as np
import spikemetrics.metrics as metrics
from spikemetrics.utils import Epoch

17:35:23 [I] klustakwik KlustaKwik2 version 0.2.6


In [2]:
workshop_path = "/Users/colehurwitz/spiketutorials/Spike_sorting_workshop_2019/"
# workshop_path = "/afs/inf.ed.ac.uk/user/s17/s1727843/spiketutorials/Spike_sorting_workshop_2019/"
recording = se.MEArecRecordingExtractor(recording_path= workshop_path + "recordings_36cells_four-tetrodes_30.0_10.0uV_20-06-2019_14_48.h5")


Could not load plane information. Assuming probe is in yz plane


In [3]:
recording = se.load_probe_file(recording, workshop_path + 'tetrode_16.prb')

In [4]:
sorting = ss.run_klusta(recording)

Klusta units are not unique! Using unique unit ids


In [5]:
metric_info = st.validation.validation_tools.get_quality_metric_data(recording, sorting, nPC=3, ms_before=1., ms_after=2., dtype=None, max_num_waveforms=np.inf, 
                                                        max_num_pca_waveforms=np.inf, save_features_props=False, verbose=False, seed=0)

spike_times = metric_info[0]
spike_clusters = metric_info[1]
amplitudes = metric_info[2]
pc_features = metric_info[3]
pc_feature_ind = metric_info[4]

In [6]:
metric_calculator = st.validation.qualitymetrics.MetricCalculator()

In [7]:
params = metric_calculator.get_default_params_dict()

In [8]:
metrics_pd = metrics.calculate_metrics(spike_times=spike_times, spike_clusters=spike_clusters, amplitudes=amplitudes, 
                                       pc_features=pc_features, pc_feature_ind=pc_feature_ind, params=params, epochs = None, cluster_ids=sorting.get_unit_ids())

Calculating isi violations
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating presence ratio
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating firing rate
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating amplitude cutoff
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating PC-based metrics
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░ 38% 

/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating silhouette score
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating drift metrics
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 


/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/spikemetrics-0.1.0-py3.7.egg/spikemetrics/metrics.py:371: RuntimeWarning: All-NaN slice encountered


In [9]:
metric_calculator.setup_all_metric_data(recording, sorting, nPC=3, ms_before=1., ms_after=2., dtype=None, max_num_waveforms=np.inf, \
                                        max_num_pca_waveforms=np.inf, save_features_props=False)


In [10]:
firing_rates = metric_calculator.compute_firing_rates()

num_spikes = metric_calculator.compute_num_spikes()

isi_violations = metric_calculator.compute_isi_violations(isi_threshold=0.0015, min_isi=0.000166, unit_ids=None, epoch_tuples=None)

presence_ratios = metric_calculator.compute_presence_ratios(unit_ids=None, epoch_tuples=None)

max_drifts, cumulative_drifts = metric_calculator.compute_drift_metrics(drift_metrics_interval_s=51, drift_metrics_min_spikes_per_interval=10, unit_ids=None, \
                                                                        epoch_tuples=None)
silhouette_scores = metric_calculator.compute_silhouette_score(unit_ids=None, epoch_tuples=None, seed=0)

isolation_distances = metric_calculator.compute_isolations_distances(num_channels_to_compare=13, max_spikes_for_unit=500, unit_ids=None, epoch_tuples=None, seed=0)

l_ratios = metric_calculator.compute_l_ratios(num_channels_to_compare=13, max_spikes_for_unit=500, unit_ids=None, epoch_tuples=None, seed=0)

d_primes = metric_calculator.compute_d_primes(num_channels_to_compare=13, max_spikes_for_unit=500, unit_ids=None, epoch_tuples=None, seed=0)

nn_hit_rates, nn_miss_rates = metric_calculator.compute_nn_metrics(num_channels_to_compare=13, max_spikes_for_unit=500, max_spikes_for_nn=10000, n_neighbors=4, unit_ids=None, epoch_tuples=None, seed=0)

 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░ 45% 

/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/spikemetrics-0.1.0-py3.7.egg/spikemetrics/metrics.py:371: RuntimeWarning: All-NaN slice encountered
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 


In [11]:
def nan_equal(a,b):
    try:
        np.testing.assert_equal(a,b)
    except AssertionError:
        return False
    return True

In [12]:
print(nan_equal(firing_rates, np.asarray(metrics_pd['firing_rate'])))
print(nan_equal(num_spikes, np.asarray(metrics_pd['num_spikes'])))
print(nan_equal(isi_violations, np.asarray(metrics_pd['isi_viol'])))
print(nan_equal(presence_ratios, np.asarray(metrics_pd['presence_ratio'])))
print(nan_equal(max_drifts, np.asarray(metrics_pd['max_drift'])))
print(nan_equal(cumulative_drifts, np.asarray(metrics_pd['cumulative_drift'])))
print(nan_equal(silhouette_scores, np.asarray(metrics_pd['silhouette_score'])))
print(nan_equal(isolation_distances, np.asarray(metrics_pd['isolation_distance'])))
print(nan_equal(l_ratios, np.asarray(metrics_pd['l_ratio'])))
print(nan_equal(d_primes, np.asarray(metrics_pd['d_prime'])))
print(nan_equal(nn_hit_rates, np.asarray(metrics_pd['nn_hit_rate'])))
print(nan_equal(nn_miss_rates, np.asarray(metrics_pd['nn_miss_rate'])))

True
True
True
True
True
True
True
True
True
True
True
True
